# 패키지, 데이터 로드

In [3]:
# 패키지 로드 
import pandas as pd 
from bs4 import BeautifulSoup  # html 파싱 
import re  # 정규표현식

In [4]:
# 데이터 로드 
file_path = '../../../data/mirae/2023BigFesta_NewsDataset_202306_5.xlsx'  
news = pd.read_excel(file_path)
news.head() # 확인 

,Unnamed: 0,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,0,202306,20230619_085100,시황공시,경제,한국투자증권(주) 상장지수증권 기타시장안내(TRUE 구리 선물 ETN),9.59,NaN,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
1,1,202306,20230619_085158,시황공시,경제,엔에이치투자증권(주) 상장지수증권 기타시장안내(QV 블룸버그 2X WTI원유선물 ETN),9.60,NH투자증권,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
2,2,202306,20230619_085702,시황공시,경제,한국투자증권(주) 상장지수증권 기타시장안내(TRUE 플래티넘 선물 ETN),9.57,NaN,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
3,3,202306,20230619_090159,시황공시,경제,메리츠증권(주) 상장지수증권 기타시장안내(메리츠 금 선물 ETN(H)),9.56,NaN,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
4,4,202306,20230619_084602,시황공시,경제,삼성증권(주) 상장지수증권 기타시장안내(삼성 인버스 2X 금 선물 ETN(H)),9.67,삼성증권,NaN,"<html>\n <head>\n <meta content=""text/html; c..."


In [5]:
news.drop('Unnamed: 0',axis = 1, inplace = True) # 불필요한 열 삭제 
news.head() # 확인 

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,202306,20230619_085100,시황공시,경제,한국투자증권(주) 상장지수증권 기타시장안내(TRUE 구리 선물 ETN),9.59,NaN,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
1,202306,20230619_085158,시황공시,경제,엔에이치투자증권(주) 상장지수증권 기타시장안내(QV 블룸버그 2X WTI원유선물 ETN),9.60,NH투자증권,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
2,202306,20230619_085702,시황공시,경제,한국투자증권(주) 상장지수증권 기타시장안내(TRUE 플래티넘 선물 ETN),9.57,NaN,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
3,202306,20230619_090159,시황공시,경제,메리츠증권(주) 상장지수증권 기타시장안내(메리츠 금 선물 ETN(H)),9.56,NaN,NaN,"<html>\n <head>\n <meta content=""text/html; c..."
4,202306,20230619_084602,시황공시,경제,삼성증권(주) 상장지수증권 기타시장안내(삼성 인버스 2X 금 선물 ETN(H)),9.67,삼성증권,NaN,"<html>\n <head>\n <meta content=""text/html; c..."


In [12]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172278 entries, 0 to 172277
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   NEWS_YM      172278 non-null  int64  
 1   DATE_TIME    172278 non-null  object 
 2   WRITER       172278 non-null  object 
 3   SUBCATEGORY  172278 non-null  object 
 4   TITLE        172273 non-null  object 
 5   IMPORTANCE   172278 non-null  float64
 6   ITEM_NAME    40405 non-null   object 
 7   TAG_LIST     132252 non-null  object 
 8   CONTENT      172278 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 11.8+ MB


# 키워드 지정
- 아래의 키워드를 뉴스 본문 html 이 저장된 변수인 'CONTENT'변수에 2번 이상 포함하지 않을 경우 엔터테인먼트와 무관한 기사로 판단하여 제거한다.

In [14]:
keywords = [
    '방시혁','하이브','에스엠','이수만','와이지','양현석','제이와이피','박진영',
    
    '뉴진스','방탄소년단','르세라핌',
    '세븐틴','투모로우바이투게더',
    '프로미스나인','나나','이현','미드낫','엔하이픈',
    '백호','강동호','황민현','예하나','성연','지코',
    '앤팀','히라테 유리나','보이넥스트도어',
    
    '블랙핑크','트레저','악뮤','악동뮤지션',
    '위너','지디','권지용',
    '젝스키스',
    
    '트와이스','엔믹스', '데이식스',
    'itzy','채령','류진','유나','예지'
    
    '스트레이키즈','투피엠','보이스토리',
    '야오천','니지유','엑스디너리 히어로즈',
    'boa',
    '에스파',
    '엑소',
    
    '샤이니','엔시티','엔시티 127','엔시티 드림',    
    '동방신기','슈퍼주니어','소녀시대','엑소'
    '레드벨벳','웨이션브이','슈퍼엠',
    '갓더비트','갓 더 비트','라이즈'

]

# 특정 단어 목록을 정규식 패턴으로 변환
keywords_pattern = '|'.join(keywords)

# 'PARSED CONTENT' 열에서 정규식 패턴을 검색하여 추출
filtered_df = news[news['CONTENT'].str.contains(keywords_pattern, na=False, flags=re.IGNORECASE, regex=True)]

# 특정 키워드가 2번 이상 포함된 행만 남기기
filtered_df = filtered_df[filtered_df['CONTENT'].apply(lambda text: sum(1 for keyword in keywords if keyword.lower() in text.lower()) >= 2)]


In [15]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 189 to 172102
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NEWS_YM      2020 non-null   int64  
 1   DATE_TIME    2020 non-null   object 
 2   WRITER       2020 non-null   object 
 3   SUBCATEGORY  2020 non-null   object 
 4   TITLE        2020 non-null   object 
 5   IMPORTANCE   2020 non-null   float64
 6   ITEM_NAME    719 non-null    object 
 7   TAG_LIST     1517 non-null   object 
 8   CONTENT      2020 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 157.8+ KB


# 데이터 저장

In [16]:
# 데이터 저장
file_path = '../../../data/News/news_1filtered_202306.xlsx'  
filtered_df.to_excel(file_path, index=False)